In [ ]:
%run graph_helper.ipynb

In [ ]:
obj_functions = [obj_disagreement, obj_polarization, obj_sum]

In [ ]:
DISAGREEMENT = 0
POLARIZATION = 1
BOTH = 2

In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
%matplotlib inline

# Pure greedy algorithm

In [ ]:
# determines if a given vertex should be set to zero or one, all else equal
def maximize_opinion(obj_fun, A, L, s, n, m, v):
    # save original value of innate opinion
    temp = s[v,0]
    
    obj = np.zeros(2)
    
    # objective if set to 0
    s[v,0] = 0
    obj[0] = obj_fun(A, L, s, n, m)
    
    # objective if set to 1
    s[v, 0] = 1
    obj[1] = obj_fun(A, L, s, n, m)
   
    # restore original opinion
    s[v,0] = temp    
    
    # pick the opinion value that yields bigger objective function value
    return (np.argmax(obj), np.max(obj))

In [ ]:
# chooses which vertex in the graph should be changed, and if it should be changed to a 0 or a 1.
def choose_greedy_vertex(obj_fun, A, L, s, n, m):
    # iterate over all the vertices that have not yet been changed
    vertices = np.where((s != 0.0) & (s != 1.0))
    
    # champion is the current best vertex, its opinion {0, 1}, and current best objective
    champion = (None, None, 0)
    for v in vertices[0]:
        # iterate through vertices and find vertex that, when changed to 0 or 1, yields the biggest objFun
        (changed_opinion, obj) = maximize_opinion(obj_fun, A, L, s, n, m, v)
        if obj >= champion[2]:
            champion = (v, changed_opinion, obj)
    
    return champion

# Innate/local heuristics

Localized innate greedy algorithm (disagreement):

We propose a computationally simple greedy algorithm for solving our problem of finding a subset of changed opinions that maximizes disagreement. For each $i$ from $1$ to $k$, we choose $v_i$ such that:

$$v_i = argmax_{v \in V - \hat{s}} \max \left(\sum_{e: u-v} w_e(1 - s_u)^2, \sum_{e: u-v} w_e s_u^2 \right) $$

In [ ]:
# determines if value of opinion at v should be set to 0 or 1 to maximize local disagreement
def maximize_local_disagreement(A, L, s, n, m, v):
    local_obj_arr = np.zeros(2)
    
    for i in range(len(L[v,])):
        if i != v:
            local_obj_arr[0] += -1*L[v, i] * s[i,0] * s[i,0]
            local_obj_arr[1] += -1*L[v, i] * (1.0 - s[i,0])*(1.0 - s[i,0])
            
            
    new_op = np.argmax(local_obj_arr)
    local_obj = local_obj_arr[new_op]
    temp = s[v, 0]
    s[v, 0] = new_op
    obj = obj_functions[DISAGREEMENT](A, L, s, n, m)
    s[v, 0] = temp
        
    return (new_op, local_obj, obj)

In [ ]:
# maximizing innate polarization only: \bar{s}^T \bar{s}
def obj_innate_polarization(op, n):
    op_mean = mean_center(op, n)
    return np.dot(np.transpose(op_mean), op_mean)[0,0]

Innate greedy algorithm (polarization):

For each $i$ from $1$ to $k$, we choose $v_i$ such that:

$$v_i = argmax_{v \in V - \hat{s}} \max \left(\bar{s}_1^T \bar{s}_1, \bar{s}_0^T \bar{s}_0 \right) $$

In [ ]:
# determines if value of opinion at v should be set to 0 or 1 to maximize innate polarization 
def maximize_innate_polarization(A, L, s, n, m, v):
    innate_obj_arr = np.zeros(2)
    
    temp = s[v, 0]
    
    s[v, 0] = 0
    innate_obj_arr[0] = obj_innate_polarization(s, len(s))
    
    s[v, 0] = 1
    innate_obj_arr[1] = obj_innate_polarization(s, len(s))
    
    new_op = np.argmax(innate_obj_arr)
    innate_obj = innate_obj_arr[new_op]
    s[v, 0] = new_op
    obj = obj_functions[POLARIZATION](A, L, s, n, m)
    s[v, 0] = temp
    
    return (new_op, innate_obj, obj)

In [ ]:
# same heuristic as above, just calculating objective function for sum of both polarization and disagreement
def maximize_innate_polarization_sum(A, L, s, n, m, v):
    innate_obj_arr = np.zeros(2)
    
    temp = s[v, 0]
    
    s[v, 0] = 0
    innate_obj_arr[0] = obj_innate_polarization(s, len(s))
    
    s[v, 0] = 1
    innate_obj_arr[1] = obj_innate_polarization(s, len(s))
    
    new_op = np.argmax(innate_obj_arr)
    innate_obj = innate_obj_arr[new_op]
    s[v, 0] = new_op
    obj = obj_functions[BOTH](A, L, s, n, m)
    s[v, 0] = temp
    
    return (new_op, innate_obj, obj)

In [ ]:
# finds the opinion node that, when changed to 0 or 1, 
# maximizes local disagreement given all else the same in the graph
def choose_innate_vertex(obj_fun, A, L, s, n, m, obj_type):
    # iterate over all the vertices that have not yet been changed
    vertices = np.where((s != 0.0) & (s != 1.0))
    
    # current best vertex, its opinion {0, 1}, "innate" objective, and objective
    champion = (None, None, 0, 0)
    
    if obj_type == DISAGREEMENT:
        for v in vertices[0]:
            (changed_opinion, local_obj, obj) = maximize_local_disagreement(A, L, s, n, m, v)
            
            if local_obj > champion[2]:
                champion = (v, changed_opinion, local_obj, obj)
                
    elif obj_type == POLARIZATION:
        for v in vertices[0]:
            (changed_opinion, innate_obj, obj) = maximize_innate_polarization(A, L, s, n, m, v)
        
            if innate_obj > champion[2]:
                champion = (v, changed_opinion, innate_obj, obj)
                
    else:
        for v in vertices[0]:
            (changed_opinion, innate_obj, obj) = maximize_innate_polarization_sum(A, L, s, n, m, v)
        
            if innate_obj > champion[2]:
                champion = (v, changed_opinion, innate_obj, obj)

    return (champion[0], champion[1], champion[3])

# Complete randomization

In [ ]:
def choose_random(obj_fun, A, L, s, n, m):
    vertices = np.where((s != 0.0) & (s != 1.0))    
        
    # randomly select vertex
    v = np.random.choice(vertices[0])
    
    # randomly set to 0 or 1
    changed_opinion = np.random.randint(0, 2)
    s[v, 0] = changed_opinion
    
    obj = obj_fun(A, L, s, n, m)
    
    return (v, changed_opinion, obj)

# Mean opinion heuristic (complete)

In [ ]:
# determines if value of opinion at v should be set to 0 or 1 to maximize objective function (rigorous)
def maximize_01(obj_fun, A, L, s, v, m):
    obj = np.zeros(2)
    
    temp = s[v, 0]
    
    s[v, 0] = 0
    obj[0] = obj_fun(A, L, s, len(s), m)
    
    s[v, 0] = 1
    obj[1] = obj_fun(A, L, s, len(s), m)

    s[v, 0] = temp
    
    return (np.argmax(obj), np.max(obj))

# choose vertex whos opinion is closest to the mean of all vertices, and set to 0 or 1 depending on what maximizes
# objective function
def choose_mean(obj_fun, A, L, s, n, m):
    vertices = np.where((s != 0.0) & (s != 1.0))
        
    avg = np.mean(s)
    v = vertices[0][np.argmin(np.absolute(s[vertices[0], 0] - avg))]
   
    (changed_opinion, obj) = maximize_01(obj_fun, A, L, s, v, m)
        
    return (v, changed_opinion, obj)

# Mean opinion heuristic (randomized)

In [ ]:
# choose the vertex whos opinion is closest to the mean of all vertices, and randomly set to 0 or 1
def choose_partial_random(obj_fun, A, L, s, n, m):
    vertices = np.where((s != 0.0) & (s != 1.0))
    
    avg = np.mean(s)
    v = vertices[0][np.argmin(np.absolute(s[vertices[0], 0] - avg))]
        
    changed_opinion = np.random.randint(0, 2)
    
    s[v, 0] = changed_opinion
    obj = obj_fun(A, L, s, n, m)
            
    return (v, changed_opinion, obj)

Consider other methods of choosing opinion to be $0$ or $1$.

$$\sum_{i = 1}^n \sum_{j = i + 1}^n w_{ij} (z_i - z_j) ((I + L)^{-1}_{ix} - (I + L)^{-1}_{jx}) $$

In [ ]:
# set vertex to 0 or 1 as to maximize an expression obtained from simplification of change in disagreement
# see above expression
def first_term_disagreement(obj_fun, A, L, s, v, m):
    z = np.dot(A, s)
    n = len(s)
    total = 0.0
    for i in range(n):
        for j in range(i + 1, n):
            total += (-L[i,j])*(z[i] - z[j])*(A[i, v] - A[j, v])
            
    if total < 0:
        s[v, 0] = 0
        obj = obj_fun(A, L, s, len(s), m)
        return(0, obj)
    else:
        s[v, 0] = 1
        obj = obj_fun(A, L, s, len(s), m)
        return(1, obj)

$$\sum_{i=1}^n \left((I + L)^{-1}_{ix} - \frac{1}{n} \right) \left(z_i - \frac{1}{n}\sum_{a = 1}^n z_a \right)$$

In [ ]:
# set vertex to 0 or 1 as to maximize an expression obtained from simplification of change in disagreement
# see above expression
def first_term_polarization(obj_fun, A, L, s, v, m):
    z = np.dot(A, s)
    n = len(s)
    total = 0.0
    for i in range(n):
        total += (A[i, v] - (1.0/n))*(z[i] - (sum(z)/n))
            
    if total < 0:
        s[v, 0] = 0
        obj = obj_fun(A, L, s, len(s), m)
        return(0, obj)
    else:
        s[v, 0] = 1
        obj = obj_fun(A, L, s, len(s), m)
        return(1, obj)

In [ ]:
def first_term_sum(obj_fun, A, L, s, v, m):
    z = np.dot(A, s)
    n = len(s)
    total = 0.0
    
    for i in range(n):
        for j in range(i + 1, n):
            total += (-L[i,j])*(z[i] - z[j])*(A[i, v] - A[j, v])
    
    for i in range(n):
        total += (A[i, v] - (1.0/n))*(z[i] - (sum(z)/n))
      
    
    if total < 0:
        s[v, 0] = 0
        obj = obj_fun(A, L, s, len(s), m)
        return(0, obj)
    else:
        s[v, 0] = 1
        obj = obj_fun(A, L, s, len(s), m)
        return(1, obj)    

In [ ]:
# a heuristic for determining if the mean vertex should be changed to a 0 or 1, depending on its neighbors
# a localized innate measure as well - may perform better since only the 0 or 1 is being changed
def neighbors_01_disagreement(obj_fun, A, L, s, v, m):
    # normalize neigbors
    weight_sum = np.sum(np.abs(L[v,]))/2
    normalized_neighbors = np.abs(L[v,])/weight_sum
    
    normalized_opinions = s[:,0]*2 - 1
    
    # compute bias function
    neighbor_bias = np.dot(normalized_neighbors, normalized_opinions) 
    - normalized_neighbors[v]*normalized_opinions[v]
    
    if neighbor_bias > 0:
        s[v, 0] = 0
        obj = obj_fun(A, L, s, len(s), m)
        return (0, obj)
    else:
        s[v, 0] = 1
        obj = obj_fun(A, L, s, len(s), m)
        return (1, obj)
    
    
def neighbors_01_polarization(obj_fun, A, L, s, v, m):
    # normalize neigbors
    weight_sum = np.sum(np.abs(L[v,]))/2
    normalized_neighbors = np.abs(L[v,])/weight_sum
    
    normalized_opinions = s[:,0]*2 - 1
    
    # compute "bias" function
    neighbor_bias = np.dot(normalized_neighbors, normalized_opinions) 
    - normalized_neighbors[v]*normalized_opinions[v]
    
    if neighbor_bias > 0:
        s[v, 0] = 1
        obj = obj_fun(A, L, s, len(s), m)
        return (1, obj)
    else:
        s[v, 0] = 0
        obj = obj_fun(A, L, s, len(s), m)
        return (0, obj)

In [ ]:
# choose vertex whose opinion is closest to the mean of all vertices, and set to 0 or 1 depending on what maximizes
# objective function
def choose_mean_set_opinion(obj_fun, A, L, s, n, m, obj_type):
    vertices = np.where((s != 0.0) & (s != 1.0))
    v = vertices[0][np.argmin(np.absolute(s[vertices[0], 0] - 0.5))]
    if obj_type == DISAGREEMENT:
        (changed_opinion, obj) = first_term_disagreement(obj_fun, A, L, s, v, m)  
    elif obj_type == POLARIZATION:
        (changed_opinion, obj) = first_term_polarization(obj_fun, A, L, s, v, m)  
    else:
        (changed_opinion, obj) = first_term_sum(obj_fun, A, L, s, v, m)
        
    return (v, changed_opinion, obj)

In [ ]:
# note: refactored, not called anywhere yet (replace)
def choose_mean_neighbors_opinion(obj_fun, A, L, s, n, m, obj_type):
    vertices = np.where((s != 0.0) & (s != 1.0))
    
    v = vertices[0][np.argmin(np.absolute(s[vertices[0], 0] - 0.5))]
    if obj_type == DISAGREEMENT:
        (changed_opinion, obj) = neighbors_01_disagreement(obj_fun, A, L, s, v, m)
    elif obj_type == POLARIZATION:
        (changed_opinion, obj) = neighbors_01_polarization(obj_fun, A, L, s, v, m)
    else:
        raise Exception("Invalid objective type.")
        
    return (v, changed_opinion, obj)

# Max (weighted) degree heuristic

In [ ]:
def degree(idx, n, L):
    # compute number of edges
    deg = 0
    for j in range(n):
        if idx != j and L[idx, j] < 0:
            deg += 1
                
    return deg

# pick the vertex that has the greatest degree
def choose_max_deg(obj_fun, A, L, s, n, m):
    vertices = np.where ((s != 0.0) & (s != 1.0))
    
    b = np.array(list(map(lambda x: degree(x, n, L), vertices[0])))
    biggest_deg = np.random.choice(np.flatnonzero(b == b.max()))
        
    v = vertices[0][biggest_deg]
    
    (changed_opinion, obj) = maximize_01(obj_fun, A, L, s, v, m)

    return (v, changed_opinion, obj)

In [ ]:
def weighted_degree(idx, n, L):
    return L[idx, idx]

# pick the vertex that has the greatest sum of weights
def choose_max_weighted_deg(obj_fun, A, L, s, n, m):
    vertices = np.where ((s != 0.0) & (s != 1.0))
    
    b = np.array(list(map(lambda x: weighted_degree(x, n, L), vertices[0])))
    biggest_deg = np.random.choice(np.flatnonzero(b == b.max()))
        
    v = vertices[0][biggest_deg]
    
    (changed_opinion, obj) = maximize_01(obj_fun, A, L, s, v, m)

    return (v, changed_opinion, obj)

# Non-adaptive heuristics (for stochastic block model)

In [ ]:
# change k/2 of community 1 to 1 and k/2 of community 2 to 0
def choose5050(obj_fun, A, L, s, n, m, k, c1, c2):
    to_zero = np.random.choice(c1, int(k/2))
    to_one = np.random.choice(c2, k - int(k/2))
    
    s[to_zero] = 1
    s[to_one] = 0
    max_obj_50 = obj_fun(A, L, s, n, m)
        
    return (to_zero, to_one, max_obj_50)

In [ ]:
# change k/4 of community 1 to 0, k/4 to 1
# change k/4 of community 2 to 0, k/4 to 1
def choose25(obj_fun, A, L, s, n, m, k, c1, c2):
    to_zero_c1 = np.random.choice(c1, int(k/4))
    to_zero_c2 = np.random.choice(c2, int(k/4))

    to_one_c1 = np.random.choice(np.setdiff1d(c1, to_zero_c1), int(k/4))
    to_one_c2 = np.random.choice(np.setdiff1d(c2, to_zero_c2), int(k/4))
    
    s[to_zero_c1] = 0
    s[to_zero_c2] = 0
    s[to_one_c1] = 1
    s[to_one_c2] = 1
    max_obj_25 = obj_fun(A, L, s, n, m)

    return (to_zero_c1, to_one_c1, to_zero_c1, to_zero_c2, max_obj_25)